### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [8]:
import gzip  # Trabalha com arquivos comprimidos no formato .gz (gzip)
import pandas as pd  # Manipulação de dados em formato de tabela (DataFrame)
import numpy as np  # Operações matemáticas e estruturas de arrays eficientes
import matplotlib.pyplot as plt  # Criação de gráficos básicos e visualizações
import seaborn as sns  # Gráficos estatísticos com estilo mais bonito (baseado no matplotlib)
from sklearn.metrics import (
    roc_auc_score,        # Métrica AUC para avaliação de classificadores
    confusion_matrix,     # Matriz de confusão para avaliação de modelos
    make_scorer,          # Criação de métricas personalizadas
    f1_score              # Métrica F1 que combina precisão e recall
)
from sklearn.linear_model import LogisticRegression  # Algoritmo de regressão logística (classificação binária)
from sklearn.compose import make_column_selector as selector  # Seleciona colunas com base em tipo ou nomes
from sklearn.impute import SimpleImputer  # Preenchimento de valores ausentes (missing data)
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # Padronização de dados e codificação categórica
from sklearn.compose import ColumnTransformer  # Aplica transformações diferentes em colunas distintas
from sklearn.pipeline import Pipeline  # Cria sequência ordenada de etapas (pipeline de modelagem)
from sklearn.datasets import fetch_openml  # Baixa datasets públicos do OpenML direto pro seu notebook
from sklearn.model_selection import train_test_split  # Divide os dados entre treino e teste

#Settings for plots
plt.rcParams['figure.figsize'] = [10, 7]  # Define o tamanho padrão dos gráficos como 10x7 polegadas
plt.rcParams['font.size'] = 15            # Define o tamanho padrão da fonte nos gráficos matplotlib
sns.set(color_codes=True)                 # Ativa uso de códigos de cor (ex: 'b' para azul) no Seaborn
sns.set(font_scale=1.5)                   # Aumenta a escala do tamanho da fonte nos gráficos do Seaborn
sns.set_palette("bright")                 # Define paleta de cores vibrantes para os gráficos do Seaborn
sns.set_style("whitegrid")                # Aplica estilo com fundo branco e linhas de grade (ideal pra visualização)

In [9]:
import automl  # Importa o pacote completo 'automl', que pode conter métodos para AutoML (automatizar experimentos de machine learning)
from automl import init  # Importa especificamente a função 'init' do pacote — geralmente usada para inicializar ou configurar o ambiente AutoML

In [10]:
dataset = fetch_openml(name = 'adult', as_frame =True)
df, y = dataset.data, dataset.target

In [11]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
0,2,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States
1,3,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States
2,2,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States
3,3,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States
4,1,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba
